# Flask-Scrapper

## *Introduction*

This project consists in a web-scrapper plus a website who catch notices from different sites, and then save them in a database and send it by email to a email list. Well, let's begin with the file **"main.py"**. First of all, let's import the necessary modules:

In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from database import add_notice, verify_date, set_today_news, verify_today, delete_today_news, get_mails, get_today_news
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

Calm down, every module will be explained in this documentation

## *Web-Scrapper*

So, let's create a web-scrapper to take the notices.We need to use the BeautifulSoup for that, this class will search the passed requirements of HTML code that we'll need. We'll use the module **"requests"** to go to the necessary pages. So we have:

In [ ]:
def search_notices():
    sites = ['site1', 'site2',
             'site3', 'site4']
    html = [requests.get(site).content for site in sites]
    notices = []
    count = 0
    today = datetime.today().utcnow().date()


OK, let's talk about the script.
First of all, I've decide to put the instructions in a function because in that way I can call the function in my flask application, so it will be very useful. 
I've created a list with the sites who I want to take the notices, and then I've created a list comprehension who saves the HTML code of all the sites I've requested. I've created too a list that will store specific excerpts in HTML codes that contain the notice that we want. The **"count"** variable will serve as a counter for the html variable, and the variable today will store the date of the day that the notice was posted.

Now, we need to make a for-loop that will format the html code and take the notices of the websites. I've used a if-elif sequence for every site that we have in "sites" variable, where we have '' in the script, just because every site have your properly way to store the notices, then we'll append in "notices" variable as string.

In [ ]:
    for site in sites:
        soup = BeautifulSoup(html[count], 'html.parser')
        if '' in site:
            notice = soup.find('a', class_="HTML_CLASS").string
            resume = soup.find('div', class_="HTML_CLASS").string
            notices.append([notice, resume])

        elif '' in site:
            notice = soup.find('span', class_="HTML_CLASS").string
            resume = soup.find('p', class_="HTML_CLASS").string
            notices.append([notice, resume])
        elif '' in site:
            notice = soup.find('h2', class_="HTML_CLASS").string
            resume = soup.find('p', class_="HTML_CLASS").string
            notices.append([notice, resume])

        elif '' in site:
            notice = soup.find('a', class_="HTML_CLASS").string
            resume = soup.find('div', class_="HTML_CLASS").string
            notices.append([notice, resume, today])

        count += 1

Now, we need to create a database, so let's go to **"database.py"**.

## *Creating a database*

To create our database, we'll use the sqlite, so let's build the basic structure:

In [ ]:
import sqlite3 as sql

conn = sql.connect('notices.db')
cursor = conn.cursor()

cursor.execute('CREATE TABLE IF NOT EXISTS notices(id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, title TEXT, '
               'resume TEXT, date DATETIME)')

cursor.execute('CREATE TABLE IF NOT EXISTS emails(id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, email TEXT)')

ok, we've created a database who contains two tables to store the notices and the user's emails. Now let's create the function that will store the notices:

In [ ]:
def add_notice(title, resume, date):
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO notices(title, resume, date) VALUES (?,?,?)', (title, resume, date))
    conn.commit()

Ok, we've created the function, but I want you to notice that we wave a problem. When we add a notice on our database, we don't have (yet) a filter that saves only the news of the day, so if we run our script many times, the notices will repeat every time and it get's worse as the days go by. Briefly, it will overload and pollute our database. So we need a function that filters the notice. Then we have:

In [ ]:
def verify_date(date):
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute("SELECT date FROM notices")
    dates = cursor.fetchall()
    verify = False
    date = str(date)
    for date_ in dates:
        if date in date_[0]:
            verify = True
            break
    return verify

in **"verify date"** function, we'll pass the today's date in **"date"** variable, select the date colum from the notices table, and scan it comparing with the passing variable **"date"**, then we have a verifier that solves our problem.

So, back to the **"main.py"** file, we have (still inside of "search notices, but out of the for-loop): 

In [ ]:
 verify = verify_date(today)
 for notice_ in notices:
    if verify:
        pass
    if not verify:
        add_notice(notice_[0], notice_[1], today)

In the script ahead, if our verify function returns true, it means that the notices of today are already in our database, so we just ignore that. But if the date are not in our database, then whe use the **"add_notice"** function to put the notices there.

Ok, I guess that now we can create the **"app.py"** to make the flask application, so let's do it:

## *Creating a Flask Application*

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from database import show_notices, save_email, search_email
from main import search_notices, send_mail
import schedule
from threading import Thread
from time import sleep

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///notices.sqlite3'


@app.route('/')
def index():
    return render_template('index.html')

Calm down, again, every import will make sense, be patient and focus on basic script for Flask

Ok, after create the basic structure for a Flask application, we can open the website using "flask run" on terminal and opening the local host. Let's create now a page to show the notices

In [ ]:
@app.route('/notices', methods=['GET', 'POST'])
def notices():
    return render_template('notices.html')

Ok, now, I guess that you'll agree with me when I say that we need a function who can take the notices in our database, so let's create her in our **"database.py"** file:

In [ ]:
def show_notices():
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM notices ORDER BY id Desc')
    notices = cursor.fetchall()
    return notices

The ahead function will select all the columns from notices table, ordered from top to bottom so you can show the news of the day first. Let's return to our flask file and change our notices function. So we have:

In [ ]:
@app.route('/notices', methods=['GET', 'POST'])
def notices():
    notices_ = show_notices()
    return render_template('notices.html', notices=notices_)

Ok, we've passed a second parameter in our render template to put the notices on our html file named **"notices.html"** by jinja, that allows python scripts in html file

Now, let's create a page who the user's can register the email:

In [ ]:
@app.route('/data')
def data():
    return render_template('data.html')

Alright, but thats not enough, now, we have to create a function to the page but with the **"post"** method, and then we need to take the email of our html file, so we have:

In [ ]:
@app.route('/data', methods=['POST'])
def data_post():
    email = request.form.get('email')

Ok, so now we have to do two things: Save the email in our database and verify if the email has already been stored in our database, thus avoiding duplicated emails. So, in our database file, we need to add:

In [ ]:
def save_email(email):
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO emails(email) VALUES (?)', (email,))
    conn.commit()

In [ ]:
def search_email(email):
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('SELECT email FROM emails')
    emails = cursor.fetchall()
    verify = False
    for email_ in emails:
        if email in email_[0]:
            verify = True
            break
    return verify

The "search_mail" has the same precept of the notice verifier, so no need for introductions, except by the fact that we'll compare our passed **"email"** with **email_[0]**, to obtain the correct format of the emails stored in database

So, going back to our **"data_post"** function in flask app, we have:

In [ ]:
def data_post():
    email = request.form.get('email')
    verify = search_email(email)
    if verify:
        pass
    else:
        save_email(email)
    return redirect(url_for('index'))

Ok, we'll return to flask application later, for now, just add in the final of your script in this file the following command:

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

Now, let's focus on send a email with the notices that we have stored. For this, we'll go to the "main.py" file and continue below of the **add_notice**, in the **"if not verify"**.

## *Sending email*

Ok, pay attention now. What we want to do it's send the notices of the day for a list of emails, but we wave a problem: We don't have a control of what news are from today, and our functions to manipulate the database maybe could help us, but I prefer to create a new table only with the notices of the day and manipulate her. So, for this, we need firstly create a new table below the others that we've already created, so, in database.py file:

In [ ]:
cursor.execute('CREATE TABLE IF NOT EXISTS today_posts(id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, title TEXT, '
               'resume TEXT, date DATETIME)')

So now, we need to create two function: one by add the news and other by verify if the news are updated:

In [ ]:
def set_today_news(title, resume, date):
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO today_posts(title, resume, date) VALUES (?,?,?)', (title, resume, date))
    conn.commit()

In [ ]:
def verify_today(date):
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('SELECT date FROM today_posts')
    date_posts = cursor.fetchall()
    date = str(date)
    is_new = False
    for date_ in date_posts:
        if date not in date_[0]:
            is_new = True
            break
    return is_new

Before we go back to the **"main.py"** script, let's define some other things:

**[1]** - In a moment, we need to get the today's news to send them, so we need to create a function to return the notices;

**[2]** - Every day, we need to delete the data stored on **"today_news"** table, to update her, so we need to delete the old news;

**[3]** - In a moment, we need to get the emails stored on our database, so we need to create a function to return them.

Knowing these things, let's create the functions:

In [ ]:
def get_today_news():
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM today_posts')
    posts = cursor.fetchall()
    return posts

In [ ]:
def delete_today_news():
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('DELETE FROM today_posts')
    conn.commit()

In [ ]:
def get_mails():
    conn = sql.connect('notices.db')
    cursor = conn.cursor()
    cursor.execute('SELECT email FROM emails')
    list_emails = []
    emails = cursor.fetchall()
    for email in emails:
        list_emails.append(email[0])
    return list_emails

Now we can finally return to the **"main.py"** file, below of the add_notice in the "if not verify".

Basically, what we need to do in the not verify it's add and verify if the news are updated or not, so it will be:

In [ ]:
        if not verify:
            add_notice(notice_[0], notice_[1], today)
            set_today_news(notice_[0], notice_[1], today)
            verify_today_post = verify_today(today)
            if not verify_today_post:
                pass
            else:
                delete_today_news()
                set_today_news(notice_[0], notice_[1], today)

Ok, we ended the **search_notices** function. Now we need to create a function that will send our mail with the news to our list of emails, so let's create and configure her:

In [ ]:
def send_mail():
    email_list = get_mails()
    email_from = os.getenv('EMAIL')
    email_password = os.getenv('PASSWORD')
    email_smtp = 'smtp.gmail.com'
    destination = email_list
    subject = 'Noticias diarias'

In the ahead script, we basically configured our email, our password, the smtp who will send our message and the email-list, so now we need to format our mail content:

In [ ]:
message = MIMEMultipart()
message['FROM'] = email_from
message['SUBJECT'] = subject
message['To'] = ", ".join(destination)

Now we need to create our text with the title and the resume of the news, one by one, so:

In [ ]:
text = ''
notices = get_today_news()
for notice in notices:
    text += f'<b>{notice[1]}</b>' + '<br>' + f'{notice[2]}' + '<br><br>'
msg_text = MIMEText(text, 'html')
message.attach(msg_text)

Now, let's just send the message:

In [ ]:
    try:
        smtp = smtplib.SMTP(email_smtp, 587)
        smtp.ehlo()
        smtp.starttls()
        smtp.ehlo()
        smtp.login(email_from, email_password)
        smtp.sendmail(email_from, destination, message.as_string())
        print('Email enviado com sucesso')
    except Exception as e:
        print(f'Falha ao enviar email: {e}')

## *Automating the sending of the messages*

Now, let's return to the **"main.py"** file.

We need to automate the message sending, for this, we'll use the schedule module, it allows you to do actions  in certain periods of time.

But it's not enough, we need to implement something that can run this script in parallel with the main script, I've chose the use of multithreading, so we have below of the app configurations:

In [ ]:
try:
    schedule.every().day.at('21:40').do(send_mail)
except TypeError:
    pass


def loop_schedule():
    while True:
        schedule.run_pending()
        sleep(1)


send = Thread(target=loop_schedule)
send.start()

And that's it! The script is ready!

by: Gabriel Chaves